In [1]:
import torch

In [2]:
!pip install torch-geometric 

     |████████████████████████████████| 184kB 4.7MB/s 
     |████████████████████████████████| 235kB 15.2MB/s 
     |████████████████████████████████| 2.2MB 19.3MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.1-cp36-none-any.whl size=308551 sha256=7562615fa27f0d513966c56cb8166a529a93bf5860a9262b1dd0a36ff6bf7cda
  Stored in directory: /root/.cache/pip/wheels/e6/25/ea/3d71d2088dccc63214fa59259dcc598ded4150a5f8b41d84ff
Successfully built torch-geometric


In [3]:
x = torch.tensor([[2, 1], [5, 6], [3, 7], [12, 0]], dtype = torch.float)
y = torch.tensor([0, 1, 0, 1], dtype = torch.float)

In [4]:
edge_index = torch.tensor([[0, 1, 2, 3, 0],
                           [1, 0, 1, 2, 3]], dtype = torch.long)

In [8]:
!pip install --verbose --no-cache-dir torch-scatter
!pip install --verbose --no-cache-dir torch-sparse
!pip install --verbose --no-cache-dir torch-cluster
!pip install torch-geometric
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Created temporary directory: /tmp/pip-ephem-wheel-cache-p5s2m1y5
Created temporary directory: /tmp/pip-req-tracker-tjg_2mai
Created requirements tracker '/tmp/pip-req-tracker-tjg_2mai'
Created temporary directory: /tmp/pip-install-ki3_125f
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-tjg_2mai'
Created temporary directory: /tmp/pip-ephem-wheel-cache-fz6z6_hc
Created temporary directory: /tmp/pip-req-tracker-tbbb6cii
Created requirements tracker '/tmp/pip-req-tracker-tbbb6cii'
Created temporary directory: /tmp/pip-install-2x10gix9
1 location(s) to search for versions of torch-sparse:
* https://pypi.org/simple/torch-sparse/
Getting page https://pypi.org/simple/torch-sparse/
Found index url https://pypi.org/simple
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /simple/torch-sparse/ HTTP/1.1" 200 1935
Analyzing links from page https://pypi.org/simple/torch-sparse/
  Found link https://files.pythonhosted.org/packages/21/a6/af5865f7bc2dc45ea789ebb35bdf

In [9]:
#create a Data object to represent a graph data

from torch_geometric.data import Data   

dataset = Data(x=x, y=y, edge_index = edge_index)
dataset

Data(edge_index=[2, 5], x=[4, 2], y=[4])

In [11]:
#create an In-Memory dataset, for the data that should fit inside the RAM
#to use the InMemory dataset class we need  functions

import torch
from torch_geometric.data import InMemoryDataset

class MyDataset(InMemoryDataset):
  def __init__(self, root, transform = None, pre_transform = None):
    super(MyDataset, self).__init__(root, transform, pre_transform)
    self.data, self.slices = torch.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return ['file1', 'file2', ...]

  @property
  def processed_file_names(self):
    return ['data.pt']

  def download(self):
    pass

  def process(self):

    data_list  = [...]

    if self.pre_filter is not None: 
      data_list = [data for data in data_list if self.pre_filter(data)]

    if self.pre_transform is not None:
      data_list = [self.pre_transform(data) for data in data_list]

    data, slices = self.collate(data_list)
    torch.save((data, slices), self.processed_paths[0])           


In [ ]:
#create a DataLoader object, to pass in the data by batches

#similar to Data object but this also has the Batch attribute which is used to denote which graph the data belongs to
#takes in the features, labels, edge indices just like in the Data obj but along with a new attr called Batch

#syntax : 

from torch_geometric.data import DataLoader

loader = DataLoader(dataset, batch_size = 512, shuffle = True)

for batch in loader:
  batch

In [14]:
#MessagePassing - VVV Important



In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [18]:
from torch_geometric.nn import MessagePassing

In [21]:
#implementation of SageConv Layer from the GraphSAGE Paper

class SageConv(MessagePassing):

  def __init__(self, in_channels, out_channels):
    super(SageConv, self).__init__(aggr = 'max') #max pool aggregator
    self.linear = torch.nn.Linear(in_channels, out_channels)
    self.activation = torch.nn.ReLU()

    #Modify as per update function requirements
    self.update_linear = torch.nn.Linear(in_channels + out_channels, in_channels, bias = False)
    self.update_activation = torch.nn.ReLU()


  def message(self, x_j):
    #shape of x_j is [E, in_channels]
    #message function is Act(wx + b), which is Act(Linear Layer)

    x_j = self.linear(x_j)
    x_j = self.activation(x_j)
    return x_j


  def update(self, aggr_out, x):
    #shape of aggr_out is [N, out_channels]
    new_emb = torch.cat([aggr_out, x], dim = 1)
    new_emb = self.update_linear(new_emb)
    new_emb = self.update_activation(new_emb)

    return new_emb  